In [1]:
from pathlib import Path
from time import time
import datetime

import numpy as np
import pandas as pd

from scipy.stats import spearmanr

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [2]:
# Global Settings

# Directories
model_dataDir = 'DATA/MODELDATA/'

# Time
START = 2020
END = 2022


# Helpers
sns.set_style('whitegrid')
np.random.seed(42)
idx = pd.IndexSlice


In [3]:
# Helper Functions
def get_trade_prices(start, stop):
    model_mdf = pd.read_csv(model_dataDir + 'ModelData.csv')
    model_mdf.rename(columns={'Unnamed: 0': 'Dates'}, inplace=True)
    model_mdf['Dates'] = pd.to_datetime(model_mdf['Dates'])
    model_mdf.set_index(['Dates', 'Coin'], inplace=True)
    #model_mdf = model_mdf.loc[:,'Open']
    #model_mdf = model_mdf.loc[idx[tickers, str(START):str(END)], 'Open']
    prices = model_mdf.unstack('Coin').sort_index().shift(-1).tz_localize('UTC')
    
    return prices


def get_predictions(predictions, prices):

    prices = prices.loc[:,'Open':'High']
    best_alpha = predictions.groupby('alpha').apply(lambda x: spearmanr(x.actuals, x.predicted)[0]).idxmax()
    predictions = predictions[predictions.alpha == best_alpha]
    predictions.index.names = ['Coin', 'Dates']
    tickers = predictions.index.get_level_values('Coin').unique()
    
    start = predictions.index.get_level_values('Dates').min().strftime('%Y-%m-%d')
    stop = (predictions.index.get_level_values('Dates').max() + pd.DateOffset(1)).strftime('%Y-%m-%d')
    prices = prices.sort_index().loc[idx[tickers, START:END], :]
    predictions = predictions.loc[predictions.alpha == best_alpha, ['predicted']]
    
    return predictions.join(prices, how='right')

In [25]:
# Import data

# Prices
trade_prices = get_trade_prices(2020, 2022)
trade_prices = trade_prices.loc[:,'Open':'Close']

# Benchmark
open_prices = trade_prices.loc[: , 'Open']
btc_bench = open_prices.bitcoin.pct_change()

# Predictions
lr_predictions = pd.read_hdf(f'{model_dataDir}lr_model_predictions.h5', 'lr/predictions')
ridge_predictions = pd.read_hdf(f'{model_dataDir}Ridge_Predictions.h5', 'ridge/predictions')
lasso_predictions = pd.read_hdf(f'{model_dataDir}Lasso_Predictions.h5', 'lasso/predictions')

# Backtest Prices and Predictions
#data = get_predictions(ridge_predictions, trade_prices)


In [26]:
trade_prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 745 entries, 2020-01-01 00:00:00+00:00 to 2022-01-14 00:00:00+00:00
Columns: 164 entries, ('Open', 'algorand') to ('Close', 'zcash')
dtypes: float64(164)
memory usage: 960.4 KB


In [29]:




best_alpha = ridge_predictions.groupby('alpha').apply(lambda x: spearmanr(x.actuals, x.predicted)[0]).idxmax()
predictions = ridge_predictions.loc[ridge_predictions.alpha == best_alpha, ['predicted']]
predictions = predictions.unstack('Coin').sort_index().shift(-1).tz_localize('UTC')

tickers = predictions.index.get_level_values('Coin').unique()
predictions = predictions.sort_index().loc[idx[tickers, START:END], :]

predictions.head()

KeyError: 'Requested level (Coin) does not match index name (Dates)'

In [28]:
trade_prices.head()

Open                                  \
Coin                       algorand   arweave basic-attention-token   
Dates                                                                 
2020-01-01 00:00:00+00:00  0.217110  0.652953              0.190072   
2020-01-02 00:00:00+00:00  0.225794  0.698436              0.185653   
2020-01-03 00:00:00+00:00  0.228031  0.668637              0.186281   
2020-01-04 00:00:00+00:00  0.234223  0.597861              0.190017   
2020-01-05 00:00:00+00:00  0.233386  0.722573              0.191317   

                                                                 \
Coin                      binancecoin      bitcoin bitcoin-cash   
Dates                                                             
2020-01-01 00:00:00+00:00   13.522027  7139.044338   201.799270   
2020-01-02 00:00:00+00:00   13.579740  7196.883716   203.727639   
2020-01-03 00:00:00+00:00   13.650140  7320.309525   221.811173   
2020-01-04 00:00:00+00:00   13.972162  7448.688305   223.863172   
2020-01-05 00:00:00+00:00   14.380112  7520.122401   236.732977   

                                                                            \
Coin                      bitcoin-cash-sv bittorrent-2   cardano chainlink   
Dates                                                                        
2020-01-01 00:00:00+00:00       95.251893     0.000280  0.032875  1.784257   
2020-01-02 00:00:00+00:00       96.466791     0.000283  0.033663  1.780386   
2020-01-03 00:00:00+00:00      104.624500     0.000289  0.033864  1.791668   
2020-01-04 00:00:00+00:00      110.612056     0.000290  0.034543  1.804323   
2020-01-05 00:00:00+00:00      115.716651     0.000296  0.035271  1.838128   

                           ...      Close                                \
Coin                       ...     monero       okb    ripple   stellar   
Dates                      ...                                            
2020-01-01 00:00:00+00:00  ...  47.084644  2.538091  0.186359  0.043809   
2020-01-02 00:00:00+00:00  ...  50.426083  2.623903  0.192295  0.045613   
2020-01-03 00:00:00+00:00  ...  51.133088  2.687003  0.194484  0.045920   
2020-01-04 00:00:00+00:00  ...  55.929731  2.706725  0.196326  0.046280   
2020-01-05 00:00:00+00:00  ...  57.802631  2.828519  0.217364  0.049974   

                                                                      \
Coin                      terra-luna     tezos theta-token      tron   
Dates                                                                  
2020-01-01 00:00:00+00:00   0.201861  1.250511    0.083924  0.012860   
2020-01-02 00:00:00+00:00   0.227293  1.281557    0.086965  0.013379   
2020-01-03 00:00:00+00:00   0.238486  1.265864    0.092759  0.013793   
2020-01-04 00:00:00+00:00   0.236526  1.264639    0.098826  0.013865   
2020-01-05 00:00:00+00:00   0.221954  1.299673    0.095617  0.014609   

                                                
Coin                        vechain      zcash  
Dates                                           
2020-01-01 00:00:00+00:00  0.005126  26.936930  
2020-01-02 00:00:00+00:00  0.005325  28.206837  
2020-01-03 00:00:00+00:00  0.005594  30.222763  
2020-01-04 00:00:00+00:00  0.005530  31.562151  
2020-01-05 00:00:00+00:00  0.005663  31.965821  

[5 rows x 164 columns]

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
btc_bench.tail()

In [ ]:
trade_prices.head()

In [ ]:
ridge_predictions.info()

In [ ]:
# Vectorized backtest

In [ ]:
# Event driven backtest with Backtestester